<a href="https://colab.research.google.com/github/Igor-ID/Fashion_adviser_telegram_bot/blob/main/preprocessing/Data_Preprocessing_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
import os
import sys
import nltk
import pandas as pd
import json
import ast
import itertools
import numpy as np
import torch
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from sklearn.preprocessing import normalize
from pandas.io.json import json_normalize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Provide a valid path to the dataset
data = '/content/drive/MyDrive/convai_vinvl_dataset.csv'
df = pd.read_csv(data)
df

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","['leather black boot', 'dark black sunglasses'...","dict_keys(['boot', 'sunglasses', 'coat', 'bag'..."
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","['gray blue suit', 'burgundy red maroon shirt'...","dict_keys(['suit', 'shirt', 'sandal', 'sunglas..."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","['tan brown dress', 'red bag', 'red heel', 'st...","dict_keys(['dress', 'bag', 'heel', 'jacket'])"
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","['blue green white dress', 'teal light blue bl...","dict_keys(['dress', 'skirt', 'shoe', 'sandal',..."
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","['hanging blue dress', 'tan brown sandal', 'wh...","dict_keys(['dress', 'sandal', 'bag', 'shirt', ..."
...,...,...,...,...,...
33370,33370.jpg,"{'hat': {'pred_score': 0.8971937298774719, 'at...","{'red,blue hat': [242.73876953125, 65.00129699...","['red blue hat', 'black and white white black ...","dict_keys(['hat', 'shoe', 'shirt', 'sweater', ..."
33371,33371.jpg,{'necklace': {'pred_score': 0.9878942370414734...,"{'gold necklace': [339.7559509277344, 226.5207...","['gold necklace', 'white dress', 'black skirt'...","dict_keys(['necklace', 'dress', 'skirt', 'hand..."
33372,33372.jpg,"{'shirt': {'pred_score': 0.8176797032356262, '...","{'white,blue shirt': [117.45890045166016, 170....","['white blue shirt', 'black and white white bl...","dict_keys(['shirt', 'sneaker', 'pant', 'tights..."
33373,33373.jpg,"{'dress': {'pred_score': 0.9423413872718811, '...","{'floral,white dress': [244.40399169921875, 14...","['floral white dress', 'gold tan brown heel', ...","dict_keys(['dress', 'heel', 'shirt', 'handbag'..."


In [4]:
# Get the 'labels_with_attributes' column from the "boudaries" column
labels_w_attr = df['boudaries(X,y,Width,Height)'].apply(ast.literal_eval)
label_w_attr = []
for i in range(len(labels_w_attr)):
  label_w_attr_key = []
  for key, value in labels_w_attr[i].items():
    label_w_attr_key.append(key)
  label_w_attr.append(label_w_attr_key)

df['labels_with_attributes'] = label_w_attr

df.head(10)

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","[leather,black boot, dark,black sunglasses, bl...","dict_keys(['boot', 'sunglasses', 'coat', 'bag'..."
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","[gray,blue suit, burgundy,red,maroon shirt, bl...","dict_keys(['suit', 'shirt', 'sandal', 'sunglas..."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","[tan,brown dress, standing,brown dress, red ba...","dict_keys(['dress', 'bag', 'heel', 'jacket'])"
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","[blue,green,white dress, teal,light blue,blue ...","dict_keys(['dress', 'skirt', 'shoe', 'sandal',..."
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","[hanging,blue dress, tan,white,brown sandal, w...","dict_keys(['dress', 'sandal', 'bag', 'shirt', ..."
5,5.jpg,"{'heel': {'pred_score': 0.26625728607177734, '...","{'tan,leather,brown heel': [277.31109619140625...","[tan,leather,brown heel, gold,beige,tan,brown ...","dict_keys(['heel', 'shirt'])"
6,6.jpg,"{'dress': {'pred_score': 0.81600022315979, 'at...","{'black dress': [49.632171630859375, 186.00764...","[black dress, black,white heel, shiny,gold,sil...","dict_keys(['dress', 'heel', 'necklace', 'hat'])"
7,7.jpg,"{'dress': {'pred_score': 0.9929578304290771, '...","{'orange,brown,white,yellow dress': [282.24893...","[orange,brown,white,yellow dress, gold,black,b...","dict_keys(['dress', 'shoe', 'shirt', 'heel'])"
8,8.jpg,"{'hat': {'pred_score': 0.9488349556922913, 'at...","{'red,orange hat': [303.0572204589844, 139.425...","[red,orange hat, black,silver necklace, green ...","dict_keys(['hat', 'necklace', 'dress', 'shoe',..."
9,9.jpg,"{'dress': {'pred_score': 0.9959445595741272, '...","{'orange,pink,red dress': [363.6910400390625, ...","[orange,pink,red dress, pink,white shoe]","dict_keys(['dress', 'shoe'])"


In [5]:
# Remove empty values from the dataframe
df = df[df['labels_with_attributes'].map(lambda d: len(d)) > 0]
df = df.reset_index(drop=True)
df

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","[leather,black boot, dark,black sunglasses, bl...","dict_keys(['boot', 'sunglasses', 'coat', 'bag'..."
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","[gray,blue suit, burgundy,red,maroon shirt, bl...","dict_keys(['suit', 'shirt', 'sandal', 'sunglas..."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","[tan,brown dress, standing,brown dress, red ba...","dict_keys(['dress', 'bag', 'heel', 'jacket'])"
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","[blue,green,white dress, teal,light blue,blue ...","dict_keys(['dress', 'skirt', 'shoe', 'sandal',..."
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","[hanging,blue dress, tan,white,brown sandal, w...","dict_keys(['dress', 'sandal', 'bag', 'shirt', ..."
...,...,...,...,...,...
32692,33370.jpg,"{'hat': {'pred_score': 0.8971937298774719, 'at...","{'red,blue hat': [242.73876953125, 65.00129699...","[red,blue hat, black and white,white,black sho...","dict_keys(['hat', 'shoe', 'shirt', 'sweater', ..."
32693,33371.jpg,{'necklace': {'pred_score': 0.9878942370414734...,"{'gold necklace': [339.7559509277344, 226.5207...","[gold necklace, white,gold necklace, hanging,b...","dict_keys(['necklace', 'dress', 'skirt', 'hand..."
32694,33372.jpg,"{'shirt': {'pred_score': 0.8176797032356262, '...","{'white,blue shirt': [117.45890045166016, 170....","[white,blue shirt, white shirt, light blue,gra...","dict_keys(['shirt', 'sneaker', 'pant', 'tights..."
32695,33373.jpg,"{'dress': {'pred_score': 0.9423413872718811, '...","{'floral,white dress': [244.40399169921875, 14...","[floral,white dress, gold,tan,brown heel, whit...","dict_keys(['dress', 'heel', 'shirt', 'handbag'..."


In [6]:
!pip install transformers
from transformers import AutoModel, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 27.0 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
# Check if we can use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [8]:
def embd(list_input):
  inputs = tokenizer(list_input, padding=True, truncation=True, return_tensors="pt")
  # Apply device in case if we use GPU
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  inputs.to(device)
  with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

  return embeddings.cpu() # output embeddings must be "cpu" tensors

In [9]:
from tqdm.notebook import tqdm
tqdm.pandas()
df['embeddings'] = df['labels_with_attributes'].progress_apply(embd)
df

  0%|          | 0/32697 [00:00<?, ?it/s]

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels,embeddings
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","[leather,black boot, dark,black sunglasses, bl...","dict_keys(['boot', 'sunglasses', 'coat', 'bag'...","[[tensor(-0.5384), tensor(-0.4717), tensor(0.8..."
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","[gray,blue suit, burgundy,red,maroon shirt, bl...","dict_keys(['suit', 'shirt', 'sandal', 'sunglas...","[[tensor(-0.8670), tensor(-0.0721), tensor(-0...."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","[tan,brown dress, standing,brown dress, red ba...","dict_keys(['dress', 'bag', 'heel', 'jacket'])","[[tensor(-0.4519), tensor(-0.1604), tensor(-0...."
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","[blue,green,white dress, teal,light blue,blue ...","dict_keys(['dress', 'skirt', 'shoe', 'sandal',...","[[tensor(-0.6090), tensor(-0.0203), tensor(-0...."
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","[hanging,blue dress, tan,white,brown sandal, w...","dict_keys(['dress', 'sandal', 'bag', 'shirt', ...","[[tensor(-0.3363), tensor(-0.1597), tensor(-0...."
...,...,...,...,...,...,...
32692,33370.jpg,"{'hat': {'pred_score': 0.8971937298774719, 'at...","{'red,blue hat': [242.73876953125, 65.00129699...","[red,blue hat, black and white,white,black sho...","dict_keys(['hat', 'shoe', 'shirt', 'sweater', ...","[[tensor(-0.7873), tensor(-0.4559), tensor(0.2..."
32693,33371.jpg,{'necklace': {'pred_score': 0.9878942370414734...,"{'gold necklace': [339.7559509277344, 226.5207...","[gold necklace, white,gold necklace, hanging,b...","dict_keys(['necklace', 'dress', 'skirt', 'hand...","[[tensor(0.0103), tensor(0.3808), tensor(-0.25..."
32694,33372.jpg,"{'shirt': {'pred_score': 0.8176797032356262, '...","{'white,blue shirt': [117.45890045166016, 170....","[white,blue shirt, white shirt, light blue,gra...","dict_keys(['shirt', 'sneaker', 'pant', 'tights...","[[tensor(0.2454), tensor(-0.5521), tensor(-0.6..."
32695,33373.jpg,"{'dress': {'pred_score': 0.9423413872718811, '...","{'floral,white dress': [244.40399169921875, 14...","[floral,white dress, gold,tan,brown heel, whit...","dict_keys(['dress', 'heel', 'shirt', 'handbag'...","[[tensor(-0.2366), tensor(0.1451), tensor(-0.7..."


In [10]:
# remove redundant columns
df = df.drop(columns=['Attributes', 'boudaries(X,y,Width,Height)', 'labels'])
df

,Name,labels_with_attributes,embeddings
0,0.jpg,"[leather,black boot, dark,black sunglasses, bl...","[[tensor(-0.5384), tensor(-0.4717), tensor(0.8..."
1,1.jpg,"[gray,blue suit, burgundy,red,maroon shirt, bl...","[[tensor(-0.8670), tensor(-0.0721), tensor(-0...."
2,2.jpg,"[tan,brown dress, standing,brown dress, red ba...","[[tensor(-0.4519), tensor(-0.1604), tensor(-0...."
3,3.jpg,"[blue,green,white dress, teal,light blue,blue ...","[[tensor(-0.6090), tensor(-0.0203), tensor(-0...."
4,4.jpg,"[hanging,blue dress, tan,white,brown sandal, w...","[[tensor(-0.3363), tensor(-0.1597), tensor(-0...."
...,...,...,...
32692,33370.jpg,"[red,blue hat, black and white,white,black sho...","[[tensor(-0.7873), tensor(-0.4559), tensor(0.2..."
32693,33371.jpg,"[gold necklace, white,gold necklace, hanging,b...","[[tensor(0.0103), tensor(0.3808), tensor(-0.25..."
32694,33372.jpg,"[white,blue shirt, white shirt, light blue,gra...","[[tensor(0.2454), tensor(-0.5521), tensor(-0.6..."
32695,33373.jpg,"[floral,white dress, gold,tan,brown heel, whit...","[[tensor(-0.2366), tensor(0.1451), tensor(-0.7..."


In [ ]:
# save dataframe to pickle. Make sure you use the correct path
df.to_pickle('/content/drive/My Drive/fashion_dataset_embeddings.pkl')

In [ ]:
df_p = pd.read_pickle('/content/drive/My Drive/fashion_dataset_embeddings.pkl')